# Práctico final de curso "Astroestadística"

    El catálogo que se usará en el presente trabajo es el "All Sky Automated Survey for SuperNovae" (ASAS-SN), primer survey óptico capaz de monitorear el cielo en su totalidad desde la Tierra, alcanzando un límite de magnitud V=18. 
    Si bien el proyecto se enfoca principalmente en detectar supernovas brillantes, también ha descubierto una gran cantidad de estrellas variables de diversos tipos (Cefeidas, RR Lyrae, Rotacionales, entre otras). 
    Para este trabajo se utilizarán las binarias eclipsantes provenientes de dicho catálogo con el objetivo de clasificarlas en detached (EA), semi-detached (EB) y binarias de contacto (EW).
    La muestra a analizar consta de 154530 objetos. Además se empleará una muestra de entrenamiento de 70000 binarias provenientes del mismo catálogo.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

In [3]:
tabla = pd.read_csv('binaries_con_clasif.csv')
muestra=pd.DataFrame(tabla)
tabla2 = pd.read_csv('train.csv')
train=pd.DataFrame(tabla2)

Las muestras cuenta con 36 variables, cuyos nombres se detallan a continuación.

In [4]:
muestra.columns

Index(['raj2000', 'dej2000', 'l', 'b', 'mean_vmag', 'amplitude', 'period',
       'variable_type', 'class_probability', 'phot_g_mean_mag',
       'e_phot_g_mean_mag', 'phot_bp_mean_mag', 'e_phot_bp_mean_mag',
       'phot_rp_mean_mag', 'e_phot_rp_mean_mag', 'bp_rp', 'parallax',
       'parallax_error', 'vt', 'dist', 'j_mag', 'e_j_mag', 'h_mag', 'e_h_mag',
       'k_mag', 'e_k_mag', 'w1_mag', 'e_w1_mag', 'w2_mag', 'e_w2_mag',
       'w3_mag', 'e_w3_mag', 'w4_mag', 'e_w4_mag', 'w1_w2', 'w3_w4'],
      dtype='object')

In [7]:
train.columns

Index(['raj2000', 'dej2000', 'l', 'b', 'mean_vmag', 'amplitude', 'period',
       'variable_type', 'class_probability', 'phot_g_mean_mag',
       'e_phot_g_mean_mag', 'phot_bp_mean_mag', 'e_phot_bp_mean_mag',
       'phot_rp_mean_mag', 'e_phot_rp_mean_mag', 'bp_rp', 'parallax',
       'parallax_error', 'vt', 'dist', 'j_mag', 'e_j_mag', 'h_mag', 'e_h_mag',
       'k_mag', 'e_k_mag', 'w1_mag', 'e_w1_mag', 'w2_mag', 'e_w2_mag',
       'w3_mag', 'e_w3_mag', 'w4_mag', 'e_w4_mag', 'w1_w2', 'w3_w4'],
      dtype='object')

Se eliminan todos los posibles valores NaN que existan en ambas tablas. Durante este paso se ignoran las columnas de error, dado que poseen muchos valores NaN, luego se decidirá cómo proseguir con ellas.

In [8]:
muestra = muestra.dropna(subset=['raj2000', 'dej2000', 'l', 'b', 'mean_vmag', 'amplitude', 'period',
       'class_probability', 'phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag','bp_rp', 'parallax','vt',
       'dist', 'j_mag','h_mag','k_mag','w1_mag', 'w2_mag', 'w3_mag','w4_mag', 'w1_w2', 'w3_w4'])

In [9]:
train = train.dropna(subset=['raj2000', 'dej2000', 'l', 'b', 'mean_vmag', 'amplitude', 'period',
       'class_probability', 'phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag','bp_rp', 'parallax','vt',
       'dist', 'j_mag','h_mag','k_mag','w1_mag', 'w2_mag', 'w3_mag','w4_mag', 'w1_w2', 'w3_w4'])

In [14]:
muestra.shape

(146707, 31)

In [15]:
train. shape

(66412, 31)

Una vez eliminados tales objetos de la muestra, nos quedamos con un conjunto de 146707 datos para clasificar y 66412 objetos en la muestra de entrenamiento.

# Análisis exploratorio de datos

Se comienza analizando la media y varianza de las variables de la muestra.

In [22]:
muestra.mean(axis=0)

/var/folders/b4/t_bhm3h53198041gt8xlqx4h0000gn/T/ipykernel_54612/1878694722.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  muestra.mean(axis=0)


mean_vmag               14.306189
amplitude                0.385155
period                   1.598928
phot_g_mean_mag         14.368941
e_phot_g_mean_mag        0.005600
phot_bp_mean_mag        14.721713
e_phot_bp_mean_mag       0.016719
phot_rp_mean_mag        13.778072
e_phot_rp_mean_mag       0.015491
bp_rp                    0.943816
parallax                 0.871614
parallax_error           0.049053
vt                      57.509610
dist                  1815.316329
j_mag                   13.019370
e_j_mag                  0.029044
h_mag                   12.677035
e_h_mag                  0.034161
k_mag                   12.578118
e_k_mag                  0.039216
w1_mag                  12.474853
e_w1_mag                 0.026297
w2_mag                  12.507695
e_w2_mag                 0.030543
w3_mag                  11.849070
e_w3_mag                 0.236301
w4_mag                   8.877702
e_w4_mag                 0.343496
w1_w2                   -0.032842
w3_w4         

In [20]:
muestra.var(axis=0)

/var/folders/b4/t_bhm3h53198041gt8xlqx4h0000gn/T/ipykernel_54612/112921858.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  muestra.var(axis=0)


mean_vmag             1.680007e+00
amplitude             6.048845e-02
period                4.831291e+01
phot_g_mean_mag       2.351514e+00
e_phot_g_mean_mag     9.882845e-06
phot_bp_mean_mag      2.342438e+00
e_phot_bp_mean_mag    8.046117e-04
phot_rp_mean_mag      2.076489e+00
e_phot_rp_mean_mag    6.188736e-04
bp_rp                 1.024425e-01
parallax              7.783338e-01
parallax_error        1.236558e-02
vt                    7.290635e+04
dist                  2.463213e+06
j_mag                 1.697720e+00
e_j_mag               1.008169e-04
h_mag                 1.705258e+00
e_h_mag               1.655764e-03
k_mag                 1.740458e+00
e_k_mag               2.112045e-03
w1_mag                1.793626e+00
e_w1_mag              1.342804e-04
w2_mag                1.816518e+00
e_w2_mag              2.679961e-04
w3_mag                8.594338e-01
e_w3_mag              2.026555e-02
w4_mag                2.587166e-01
e_w4_mag              2.196436e-02
w1_w2               

    Es necesario estandarizar las variables dado que algunas tienen valores muy elevados (dist, vt) respecto al resto y podrían dominar las componentes principales al hacer el análisis PCA